
uploud the data

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import math
import statsmodels.api as sm

from random import*
from google.colab import drive
from scipy.stats import multivariate_normal
%matplotlib inline

from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, GridSearchCV, KFold,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression




#Mount Google Drive
drive.mount('/content/drive')

#Read the CSV file into a DataFrame
Xy_train = pd.read_csv("/content/drive/Shareddrives/ML/חלק ב/AirlineSatisfaction Dataset/Xy_train.csv")


Pre-Processing

In [ ]:

train_data =Xy_train

# Remove unwanted class
train_data["Class"][train_data["Class"] == "IT IS SO BORING WORKING IN AN AIRPORT'S DESK OH MY GODDDDD"] = None
train_data["Class"][train_data["Class"] == 'Unknown'] = None


for column in train_data[:-1].columns:train_data[column] = train_data[column].fillna(np.random.choice(train_data[column][~train_data[column].isna()]))

# Encode categorical variables
categorical_columns = train_data.select_dtypes(include=['object', 'category']).columns
label_encoders = {}
for column in categorical_columns:
    le = LabelEncoder()
    train_data[column] = le.fit_transform(train_data[column])
    print(train_data[column])
    label_encoders[column] = le

print('______________________________________________________________________________________________________________________-')
for column in train_data.columns: print(f"column : {column} -> {list(train_data[column].unique())}")

fit Logistic Regression

In [ ]:

# Load the data
data = train_data

# If the target variable is within the same file, separate it from features
y = data['satisfaction']  # Replace 'target' with the actual target column name
X = data.drop(columns=['satisfaction'])

# Identifying categorical and numerical columns
categorical_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class']
numerical_cols = X.columns.difference(categorical_cols)

# Define transformers for preprocessing
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Preprocess the data
X_preprocessed = preprocessor.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.1, random_state=42)

# Define and train the logistic regression model
model = LogisticRegression(max_iter=1000,penalty = None,solver='saga',)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = f1_score(y_test, y_pred)
print(f'Test accuracy: {accuracy}')

KFold



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, KFold
import numpy as np

# הגדרת רשת הפרמטרים (param_grid) עבור Logistic Regression
param_grid = {
  # 'C': np.logspace(-4, 4, 20),
  'penalty': ['none'],
  'solver': ['saga'],
  'max_iter': [1000]
}

scoring = 'f1'
grid_search = {}

for cv in [10]:
  kfold = KFold(n_splits=cv, shuffle=True, random_state=123)
  grid_search[cv] = GridSearchCV(estimator=LogisticRegression(random_state=42),
                                 param_grid=param_grid,
                                 refit=True,
                                 cv=kfold,
                                 verbose=1,
                                 scoring=scoring)

  grid_search[cv].fit(X_train, y_train)

# תוצאות רשת החיפוש
for cv in grid_search:
    print(f"CV: {cv}")
    print("Best parameters found: ", grid_search[cv].best_params_)
    print("Best F1 score: ", grid_search[cv].best_score_)


k fold plot

In [ ]:
results=[]
for key , value in grid_search.items():
  best_model = value.best_estimator_
  print('k :' , key ,value.best_params_, '\n')
  print('k :' , key ,scoring, "_score : ",value.best_score_ ,best_model.get_params(), '\n')
  # pd.DataFrame(value.cv_results_)
  results.append({'cv': key,'test_acc': value.best_score_})

# Convert results to DataFrame
res = pd.DataFrame(results)

# Plot the results
plt.figure(figsize=(13, 4))
plt.plot(res['cv'], res['test_acc'], marker='o', markersize=4)
plt.legend(['Train accuracy', f'test_{scoring}'])
plt.xlabel('cv')
plt.ylabel(f'Accuracy_{scoring}')
plt.title(f'cv vs Max Depth Accuracy_{scoring}')
plt.grid(True)
plt.show()